# Use AI classifier to spot events

We will see how a trained AI on simulation data is able to spot a dyfunction, here namely a broken fan, in operation data.

### Load the AI model

In [1]:
import numpy as np
import pandas as pd
from joblib import load

# load the already trained classifier and 
classifier = load('data/broken_fan_classifier.sk-learn')
classifier_mean = pd.read_csv("data/broken_fan_classifier_mean.csv")
classifier_std = pd.read_csv("data/broken_fan_classifier_std.csv")

### Load the operation data

In [2]:
# load the operation data to check for borken fan event
data = pd.read_csv("data/cpu_hot_day_intensive_use_broken.csv")
# do data normalization
classifier_data = (data[["fan.T_air", "T_cpu", "fan.tension"]] - classifier_mean.loc[0]) / classifier_std.loc[0]
classifier_data.T

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
T_cpu,-2.859732,-2.368006,-1.776037,-1.325597,-0.853352,-0.387675,0.037213,0.539365,0.828324,1.150726,...,4.353124,4.190347,3.981272,3.802590,3.641219,3.491789,3.377276,3.246776,3.097929,2.936554
fan.T_air,2.903702,2.914636,2.847024,2.941271,2.922106,2.903610,2.858302,2.844266,2.837582,2.859503,...,2.852302,2.914785,2.923996,2.848719,2.887518,2.929714,2.904653,2.933990,2.884613,2.890936
fan.tension,-0.662428,-0.662428,-0.662428,-0.662428,-0.662428,-0.662428,0.785637,0.785637,0.785637,0.785637,...,2.233703,2.233703,2.233703,2.233703,2.233703,2.233703,2.233703,2.233703,2.233703,2.233703


### Perform the checks

In [3]:
# use the AI classifier to predict nominal vs broken fan
diag = pd.DataFrame(np.stack((classifier_data.index, classifier.predict(classifier_data.values)), axis=1), columns=["time", "fan_working_diagnostic"])

# also get associated statistics
classifier_proba =  pd.DataFrame(np.hstack((np.arange(0, 31, 1).reshape(-1, 1), classifier.predict_proba(classifier_data.values))), columns=["time", "non_working_proba", "working_proba"])
non_working_proba = classifier_proba[["time", "non_working_proba"]].rename(columns={"non_working_proba": "proba"})
working_proba = classifier_proba[["time", "working_proba"]].rename(columns={"working_proba": "proba"})

In [4]:
from twiinit_demos.cpu.utils.plot import plot_recorders

plot_recorders({"diag": diag,
                "working_diag_proba": working_proba,
                "non_working_diag_proba": non_working_proba,
               },
               [[("time", "fan_working_diagnostic"), ("time", "proba")]],
               width=800, height=600, legend = {'x': 0.75, 'y': 0.25, 'font': {'size': 20}, 'orientation': 'h', 'xanchor': 'center'})

    'data': [{'mode': 'markers',
              'name': 'diag',
  …